<a href="https://colab.research.google.com/github/maxim920/Machine-Learning/blob/main/Kosenko_7_2_4_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:

!pip install scikit-surprise

import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import cross_validate, GridSearchCV

data = Dataset.load_builtin('ml-100k')

In [33]:
# Налаштування параметрів SVD через GridSearchCV
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.05]
}

# Виконуємо підбір параметрів
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# Виводимо найкращі параметри
print("Найкращі параметри:", gs.best_params['mae'])


Найкращі параметри: {'n_factors': 150, 'lr_all': 0.005, 'reg_all': 0.05}


In [34]:
# Оцінка моделі з найкращими параметрами
algo = gs.best_estimator['mae']
cross_validate(algo, data, measures=['MAE'], cv=5, verbose=True)


Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7348  0.7373  0.7417  0.7322  0.7335  0.7359  0.0034  
Fit time          2.04    1.91    1.95    1.95    2.93    2.16    0.39    
Test time         0.14    0.14    0.33    0.14    0.38    0.23    0.11    


{'test_mae': array([0.73483939, 0.73732966, 0.74174768, 0.73221995, 0.73346443]),
 'fit_time': (2.0377752780914307,
  1.9131016731262207,
  1.9494757652282715,
  1.9503557682037354,
  2.92537522315979),
 'test_time': (0.14177417755126953,
  0.13965940475463867,
  0.33221864700317383,
  0.1434929370880127,
  0.3783528804779053)}

In [35]:
# Створюємо trainset
trainset = data.build_full_trainset()
algo.fit(trainset)

In [36]:
# Функція для отримання рекомендацій
def get_top_n_recommendations(algo, trainset, user_id, n=10):
    # Отримуємо список усіх фільмів
    all_items = trainset.all_items()

    # Отримуємо рейтинги для всіх фільмів
    user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
    rated_items = [item[0] for item in user_ratings]

    recommendations = []
    for item_id in all_items:
        if item_id not in rated_items:
            est = algo.predict(user_id, trainset.to_raw_iid(item_id)).est
            recommendations.append((item_id, est))

    # Сортуємо за передбачуваним рейтингом
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:n]


In [40]:

user_id = str(345)
top_n = get_top_n_recommendations(algo, trainset, user_id, n=10)

# Виводимо рекомендації
print("Рекомендації для користувача:", user_id)
for movie_id, rating in top_n:
    print(f"Фільм ID: {movie_id}, Прогнозований рейтинг: {rating:.2f}")

Рекомендації для користувача: 345
Фільм ID: 180, Прогнозований рейтинг: 4.69
Фільм ID: 277, Прогнозований рейтинг: 4.65
Фільм ID: 189, Прогнозований рейтинг: 4.60
Фільм ID: 58, Прогнозований рейтинг: 4.59
Фільм ID: 174, Прогнозований рейтинг: 4.59
Фільм ID: 240, Прогнозований рейтинг: 4.58
Фільм ID: 612, Прогнозований рейтинг: 4.57
Фільм ID: 36, Прогнозований рейтинг: 4.55
Фільм ID: 538, Прогнозований рейтинг: 4.54
Фільм ID: 78, Прогнозований рейтинг: 4.53


Висновки

В результаті оцінки алгоритму SVD було виявлено, що найкращими параметрами є: n_factors = 150, lr_all = 0.005, і reg_all = 0.05. Середня абсолютна помилка (MAE) на тестовій вибірці склала 0.7359 з невеликою стандартною девіацією 0.0034, що вказує на стабільність моделі. Час навчання варіювався від 1.91 до 2.93 секунд, а час тестування від 0.14 до 0.38 секунд, що є прийнятним для даного алгоритму.

Для користувача з ID 345 система рекомендує наступні фільми з прогнозованими рейтингами:

    Фільм ID: 180, рейтинг: 4.69
    Фільм ID: 277, рейтинг: 4.65
    Фільм ID: 189, рейтинг: 4.60
    Фільм ID: 58, рейтинг: 4.59
    Фільм ID: 174, рейтинг: 4.59
    Фільм ID: 240, рейтинг: 4.58
    Фільм ID: 612, рейтинг: 4.57
    Фільм ID: 36, рейтинг: 4.55
    Фільм ID: 538, рейтинг: 4.54
    Фільм ID: 78, рейтинг: 4.53

Ці рекомендації свідчать про те, що модель може надати користувачеві цікаві фільми, базуючись на його вподобаннях.